In [1]:
import numpy as np
import torch
from hifigan import HiFiGAN, HiFiGANConfig, HiFiGANAudioConfig
from model import Tacotron2, Tacotron2Config, Tacotron2TrainState, Tacotron2Trainer
from dataset import Tacotron2Dataset, Tacotron2AudioConfig, Tacotron2SpeakerDataset

2023-08-12 06:30:22.015213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 06:30:22.738485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hifigan = HiFiGAN.load("../hifigan_model_1200000_0.3101.pt", device=device)
tacotron2 = Tacotron2.load("./output/checkpoints/t7/model_450000_0.2514.pt", device=device)

In [11]:
HiFiGAN.load("../z/z/checkpoints/17/model_best_0.2941.pt", device=device).train_state.epoch

225

In [14]:
speaker_dataset = Tacotron2SpeakerDataset("/mnt/ramdisk/5474_G2A1E7_YSH")

In [5]:
import matplotlib
from matplotlib import pyplot as plt
from IPython.display import display, Audio

def plot(mel, attn):
    fig, axs = plt.subplots(2, 1)
    im1 = axs[0].imshow(mel, aspect="auto", origin="lower")
    axs[0].set_title("Original Mel")
    fig.colorbar(im1, ax=axs[0])

    im2 = axs[1].imshow(attn, aspect="auto", origin="lower")
    axs[1].set_title("Attention")
    axs[1].set_xlabel("Audio timestep")
    axs[1].set_ylabel("Text timestep")
    axs[1].set_xlim(0, attn.shape[1])
    axs[1].set_ylim(0, attn.shape[0])
    fig.colorbar(im2, ax=axs[1])

    fig.tight_layout()

    return fig

In [ ]:
mel, audio, attn = tacotron2(
    inputs="프론트는 이번에 웹소켓 쓰는게 낫지 않아?",
    speakers=speaker_dataset.embeds(),
    vocoder=hifigan,
    include_attention=True,
)

# write("test.wav", 44100, (audios[0].cpu().numpy().squeeze() * 32768.0).astype(np.int16))
audio = audio.cpu().numpy().squeeze()
display(Audio(audio, rate=44100))
plot(mel.cpu().numpy().squeeze().T, attn.cpu().numpy().squeeze()).show()